# Load SIFT-annotated VCF and filter variants to keep deleterious with high effects  

In [1]:
# Annotations impacts definitions
# https://pcingola.github.io/SnpEff/snpeff/inputoutput/

# How many HIGH, MODERATE, LOW, MODIFIER variants in our set of data?

In [2]:
# Load part of the modules
import sys
from cyvcf2 import VCF
import csv

## 1. Load the VCF file in Python dataframe

In [3]:
#!/usr/bin/env python3
"""
Load a VCF (including INFO annotations and per‑sample genotypes) into a
pandas DataFrame called `vcf_an`.

Resulting columns:
    CHROM, POS, ID, REF, ALT, QUAL, FILTER,
    <INFO_key_1>, <INFO_key_2>, …,
    <sample1>_<FORMAT_subfield>, <sample2>_<FORMAT_subfield>, …
"""

import sys
from pathlib import Path
import pandas as pd

def parse_info(info_str: str) -> dict:
    """
    Turn the INFO string (semicolon separated key=value pairs) into a dict.
    Keys without a value (e.g., FLAG fields) get the boolean True.
    """
    info_dict = {}
    for entry in info_str.split(";"):
        if "=" in entry:
            k, v = entry.split("=", 1)
            info_dict[k] = v
        elif entry:               # flag without value
            info_dict[entry] = True
    return info_dict


def expand_genotypes(format_str: str, sample_strs: list, sample_names: list) -> dict:
    """
    Given the FORMAT column (e.g., "GT:AD:DP") and the list of per‑sample
    strings, return a flat dict where each key is
        <sample>_<subfield>
    and the value is the corresponding entry (or NaN if missing).
    """
    subfields = format_str.split(":")
    geno_dict = {}
    for name, sample in zip(sample_names, sample_strs):
        parts = sample.split(":")
        # Pad missing subfields with None so zip never drops trailing empties
        if len(parts) < len(subfields):
            parts += [None] * (len(subfields) - len(parts))
        for sf, val in zip(subfields, parts):
            key = f"{name}_{sf}"
            geno_dict[key] = val if val != "." else None
    return geno_dict


def load_vcf(vcf_path: Path) -> pd.DataFrame:
    """
    Parse the VCF and return a DataFrame named `vcf_an`.
    """
    records = []                     # list of dicts, one per variant
    with vcf_path.open() as fin:
        for line in fin:
            line = line.rstrip("\n")
            # ---------------------------------------------------------
            # 1️⃣  Skip meta‑information lines (start with ##)
            # ---------------------------------------------------------
            if line.startswith("##"):
                continue

            # ---------------------------------------------------------
            # 2️⃣  Header line – discover column names and sample IDs
            # ---------------------------------------------------------
            if line.startswith("#"):
                header = line.lstrip("#").split("\t")
                # Fixed VCF columns (always present)
                fixed_cols = ["CHROM", "POS", "ID", "REF", "ALT",
                              "QUAL", "FILTER", "INFO", "FORMAT"]
                # Anything after FORMAT are sample columns
                sample_names = header[len(fixed_cols):]
                col_index = {name: i for i, name in enumerate(header)}
                continue

            # ---------------------------------------------------------
            # 3️⃣  Variant line – split into fields
            # ---------------------------------------------------------
            fields = line.split("\t")
            # Core fields
            rec = {
                "CHROM": fields[col_index["CHROM"]],
                "POS":   int(fields[col_index["POS"]]),
                "ID":    fields[col_index["ID"]],
                "REF":   fields[col_index["REF"]],
                "ALT":   fields[col_index["ALT"]],
                "QUAL":  fields[col_index["QUAL"]],
                "FILTER":fields[col_index["FILTER"]],
            }

            # ---------------------------------------------------------
            # 4️⃣  INFO – expand into separate columns
            # ---------------------------------------------------------
            info_str = fields[col_index["INFO"]]
            rec.update(parse_info(info_str))

            # ---------------------------------------------------------
            # 5️⃣  GENOTYPES – expand per‑sample fields
            # ---------------------------------------------------------
            fmt = fields[col_index["FORMAT"]]
            sample_strs = [fields[col_index[name]] for name in sample_names]
            rec.update(expand_genotypes(fmt, sample_strs, sample_names))

            records.append(rec)

    # -----------------------------------------------------------------
    # Build the DataFrame – pandas will align all keys automatically
    # -----------------------------------------------------------------
    vcf_an = pd.DataFrame.from_records(records)

    # Optional: set a sensible order (fixed columns first, then INFO, then genotypes)
    fixed_order = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER"]
    other_cols = [c for c in vcf_an.columns if c not in fixed_order]
    vcf_an = vcf_an[fixed_order + sorted(other_cols)]

    return vcf_an


def main():
    if len(sys.argv) != 2:
        sys.stderr.write(
            "Usage: python load_vcf.py <path/to/your.vcf>\n"
        )
        sys.exit(1)

    vcf_path = Path(sys.argv[1])
    if not vcf_path.is_file():
        sys.stderr.write(f"❌ File not found: {vcf_path}\n")
        sys.exit(1)






if __name__ == "__main__":
    main()

❌ File not found: --f=/run/user/1000/jupyter/runtime/kernel-v38f31b33054da6af88b128596da71c626a218092c.json


SystemExit: 1

/home/thibauld/.pyenv/versions/3.10.14/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Now that the load function is defined, we load the VCF file.

vcf_path_str = "/home/thibauld/Documents/Bioinformatics/Deleterious_alleles_pipeline/Deleterious_alleles_PNG/conserved_SNPs/filtered_in_regions.vcf"

# Convert to Path objects
vcf_path = Path(vcf_path_str)

# Put the table in an object
vcf_an = load_vcf(vcf_path)
vcf_an

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,AC,AF,AN,...,InbreedingCoeff,LOF,MLEAC,MLEAF,MQ,MQRankSum,NMD,QD,ReadPosRankSum,SOR
0,scaffold11,2133652,.,G,"C,GTCA,A",2476.74,.,"2,1,1","0.005263,0.002632,0.002632",380,...,0.4341,NaN,"6,3,3","0.016,0.007895,0.007895",60,0,NaN,24.96,0.361,4.244
1,scaffold11,5971895,.,T,A,5133.71,.,1,0.002632,380,...,0.565,NaN,1,0.002632,60,0,NaN,12.34,1.24,0.705
2,scaffold11,6230609,.,A,C,5654.54,.,1,0.002632,380,...,0.5753,NaN,1,0.002632,60,0,NaN,10.45,0.764,0.436
3,scaffold11,6230610,.,T,C,2356.86,.,2,0.005263,380,...,0.6067,NaN,3,0.007895,60,NaN,NaN,28.24,NaN,0.931
4,scaffold11,6230612,.,G,"GACCTAC,A",69763.9,.,"1,5","0.002632,0.013",380,...,0.5922,(Bma005980.2|null.4702|1|1.00),"1,6","0.002632,0.016",60,0,(Bma005980.2|null.4702|1|1.00),34.61,0.945,0.678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,scaffold3,4691596,.,TATC,T,327.03,.,1,0.002632,380,...,0.4593,NaN,3,0.007895,60,0,NaN,14.87,-0.167,0.368
418,scaffold3,4691597,.,ATCTCAAAACATGAAGACATG,"A,*,ACTCAAAACATGAAGACATG",811.5,.,"1,1,2","0.002632,0.002632,0.005263",380,...,0.4557,NaN,"3,3,5","0.007895,0.007895,0.013",60,0,NaN,13.09,2.62,0.629
419,scaffold3,4691599,.,C,"*,T",524.95,.,"2,1","0.005263,0.002632",380,...,0.46,NaN,"5,3","0.013,0.007895",60,NaN,NaN,10.29,NaN,0.049
420,scaffold3,6043092,.,T,"G,A",3726.32,.,"1,2","0.002632,0.005263",380,...,0.5967,NaN,"1,2","0.002632,0.005263",60,0,NaN,13.07,0.705,0.896


In [ ]:
# Show headers of the table
print(vcf_an.head(0))

Empty DataFrame
Columns: [CHROM, POS, ID, REF, ALT, QUAL, FILTER, AC, AF, AN, ANN, B.S.Pet.102_AD, B.S.Pet.102_DP, B.S.Pet.102_GQ, B.S.Pet.102_GT, B.S.Pet.102_PGT, B.S.Pet.102_PID, B.S.Pet.102_PL, B.S.Pet.102_PS, B.S.Pet.104_AD, B.S.Pet.104_DP, B.S.Pet.104_GQ, B.S.Pet.104_GT, B.S.Pet.104_PGT, B.S.Pet.104_PID, B.S.Pet.104_PL, B.S.Pet.104_PS, B.S.Pet.105_AD, B.S.Pet.105_DP, B.S.Pet.105_GQ, B.S.Pet.105_GT, B.S.Pet.105_PGT, B.S.Pet.105_PID, B.S.Pet.105_PL, B.S.Pet.105_PS, B.S.Pet.127_AD, B.S.Pet.127_DP, B.S.Pet.127_GQ, B.S.Pet.127_GT, B.S.Pet.127_PGT, B.S.Pet.127_PID, B.S.Pet.127_PL, B.S.Pet.127_PS, B.S.Pet.128_AD, B.S.Pet.128_DP, B.S.Pet.128_GQ, B.S.Pet.128_GT, B.S.Pet.128_PGT, B.S.Pet.128_PID, B.S.Pet.128_PL, B.S.Pet.128_PS, B.S.Pet.129_AD, B.S.Pet.129_DP, B.S.Pet.129_GQ, B.S.Pet.129_GT, B.S.Pet.129_PGT, B.S.Pet.129_PID, B.S.Pet.129_PL, B.S.Pet.129_PS, B.S.Pet.156_AD, B.S.Pet.156_DP, B.S.Pet.156_GQ, B.S.Pet.156_GT, B.S.Pet.156_PGT, B.S.Pet.156_PID, B.S.Pet.156_PL, B.S.Pet.156_PS, B.S.Pet

In [ ]:
# Where are the annotations? What is in the ANN column?
vcf_an["ANN"]

0      A|upstream_gene_variant|MODIFIER|Bma030423|nul...
1      A|missense_variant|MODERATE|Bma005962.1|null.4...
2      C|missense_variant|MODERATE|Bma005980.2|null.4...
3      C|missense_variant|MODERATE|Bma005980.2|null.4...
4      GACCTAC|stop_gained&disruptive_inframe_inserti...
                             ...                        
417    T|downstream_gene_variant|MODIFIER|Bma016853.1...
418    ACTCAAAACATGAAGACATG|downstream_gene_variant|M...
419    T|downstream_gene_variant|MODIFIER|Bma016853.1...
420    A|intergenic_region|MODIFIER|Bma030990-Bma0309...
421    A|frameshift_variant|HIGH|Bma014354.2|null.109...
Name: ANN, Length: 422, dtype: object

In [ ]:
# We want to have these info separated into columns, let's do it.
# First we make the function

import pandas as pd
import sqlite3
from pathlib import Path

# ------------------------------------------------------------------
# The order of fields in the ANN column is defined by the VCF header:
#   Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID |
#   Feature_Type | Feature_ID | Transcript_BioType | Rank |
#   HGVS.c | HGVS.p | cDNA.pos / cDNA.length |
#   CDS.pos / CDS.length | AA.pos / AA.length | Distance |
#   ERRORS / WARNINGS / INFO
# ------------------------------------------------------------------
ANN_FIELDS = [
    "Allele",
    "Annotation",
    "Annotation_Impact",
    "Gene_Name",
    "Gene_ID",
    "Feature_Type",
    "Feature_ID",
    "Transcript_BioType",
    "Rank",
    "HGVS_c",
    "HGVS_p",
    "cDNA_pos_len",
    "CDS_pos_len",
    "AA_pos_len",
    "Distance",
    "Errors_Warnings_Info"
]

def split_ann(ann_str: str) -> dict:
    """
    Turn a single ANN string into a dict mapping each ANN field name → value.
    If the string is missing ('.' or empty) we return NaN for every field.
    """
    if not ann_str or ann_str == ".":
        # Return a dict with all fields set to None (pandas will become NaN)
        return {fld: None for fld in ANN_FIELDS}

    # Some VCFs can contain *multiple* annotations separated by commas.
    # Here we keep only the **first** annotation (the most common convention).
    # If you need all of them you could explode the DataFrame later.
    first_ann = ann_str.split(",")[0]

    parts = first_ann.split("|")
    # Pad missing trailing fields so zip never drops them
    if len(parts) < len(ANN_FIELDS):
        parts += [""] * (len(ANN_FIELDS) - len(parts))

    return dict(zip(ANN_FIELDS, parts))

In [ ]:
# Then we use the function to expand the ANN column into separate fields

# Apply the splitter row‑wise; pandas will broadcast the dict into new columns
ann_expanded = vcf_an["ANN"].apply(split_ann).apply(pd.Series)

# Merge the new columns back into the original DataFrame
vcf_an = pd.concat([vcf_an.drop(columns=["ANN"]), ann_expanded], axis=1)

# Quick sanity check – show the first few rows with the new columns
print("\n=== First 5 rows after ANN expansion ===")
print(vcf_an.head())


=== First 5 rows after ANN expansion ===
        CHROM      POS ID REF        ALT     QUAL FILTER     AC  \
0  scaffold11  2133652  .   G   C,GTCA,A  2476.74      .  2,1,1   
1  scaffold11  5971895  .   T          A  5133.71      .      1   
2  scaffold11  6230609  .   A          C  5654.54      .      1   
3  scaffold11  6230610  .   T          C  2356.86      .      2   
4  scaffold11  6230612  .   G  GACCTAC,A  69763.9      .    1,5   

                           AF   AN  ...   Feature_ID Transcript_BioType  Rank  \
0  0.005263,0.002632,0.002632  380  ...    Bma030423     protein_coding         
1                    0.002632  380  ...  Bma005962.1     protein_coding   4/4   
2                    0.002632  380  ...  Bma005980.2     protein_coding  7/10   
3                    0.005263  380  ...  Bma005980.2     protein_coding  7/10   
4              0.002632,0.013  380  ...  Bma005980.2     protein_coding  7/10   

               HGVS_c                    HGVS_p cDNA_pos_len CDS_pos

## 2. Discriminate the High-impact variants

In [ ]:
# This will be shown in the column below
vcf_an["Annotation_Impact"]

0      MODIFIER
1      MODERATE
2      MODERATE
3      MODERATE
4          HIGH
         ...   
417    MODIFIER
418    MODIFIER
419    MODIFIER
420    MODIFIER
421        HIGH
Name: Annotation_Impact, Length: 422, dtype: object

In [ ]:
# Number of distinct (unique) values
# --------------------------------------------------------------
n_unique = vcf_an["Annotation_Impact"].nunique(dropna=True)
print(f"Number of distinct Annotation_Impact values (excluding NaN): {n_unique}")

# Frequency table – each value and its count
# --------------------------------------------------------------
freq_tbl = vcf_an["Annotation_Impact"].value_counts(dropna=False)  # keep NaN as a category if you like
print("\nFrequency of each Annotation_Impact value:")
print(freq_tbl)

Number of distinct Annotation_Impact values (excluding NaN): 4

Frequency of each Annotation_Impact value:
Annotation_Impact
MODIFIER    287
MODERATE     67
LOW          47
HIGH         21
Name: count, dtype: int64


In [ ]:
# We write the DataFrame to a CSV file to be able to check all the columns (comma‑separated, with a header line)
vcf_an.to_csv("vcf_an.csv", index=False)   # index=False omits the pandas row index
print("CSV written to vcf_an.csv")

✅ CSV written to vcf_an.csv
